In [18]:
import chardet
import pandas as pd

# 检测文件编码
with open('Pokemon.csv', 'rb') as f:
    result = chardet.detect(f.read())

# 使用检测到的编码加载数据集
df = pd.read_csv('Pokemon.csv', encoding=result['encoding'])

# 1. 删除最后两行无意义数据
rows, columns = df.shape
df = df[:-2]
print('删除了最后两行无意义数据。')

# 2. 处理除 Type 2 列和 Total 列外其他列的空白值
columns_to_check = [col for col in df.columns if col not in ['Type 2', 'Total']]
nan_rows = df[columns_to_check].isnull().any(axis=1)
nan_count = nan_rows.sum()
df = df[~nan_rows]
print(f'发现{nan_count}条除 Type 2 列和 Total 列外其他列存在空白值的异常数据，已删除。')

# 3. 检测并处理数据集中的重复值
duplicate_rows = df[df.duplicated()]
if not duplicate_rows.empty:
    duplicate_count = duplicate_rows.shape[0]
    df = df.drop_duplicates()
    print(f'发现{duplicate_count}条重复数据，已删除。')
else:
    print('数据集中不存在重复值。')

# 4. 清空 Type 2 列中的异常数值
valid_type2_values = ['Grass', 'Poison', 'Fire', 'Flying', 'Water', 'Bug', 'Normal', 'Electric', 'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Steel', 'Ice', 'Ghost', 'Dragon', 'Dark']
type2_anomaly_count = df[~df['Type 2'].isin(valid_type2_values) & df['Type 2'].notnull()].shape[0]
df['Type 2'] = df['Type 2'].where(df['Type 2'].isin(valid_type2_values), None)
print(f'发现{type2_anomaly_count}条 Type 2 列的异常数据，已将异常值清空。')

# 5. 将 Attack、Defense、Sp. Atk、Speed 列转换为数值类型
df['Attack'] = pd.to_numeric(df['Attack'], errors='coerce')
df['Defense'] = pd.to_numeric(df['Defense'], errors='coerce')
df['Sp. Atk'] = pd.to_numeric(df['Sp. Atk'], errors='coerce')
df['Speed'] = pd.to_numeric(df['Speed'], errors='coerce')

# 6. 删除 Attack 属性中大于 200 的异常值
attack_anomaly_count = df[df['Attack'] > 200].shape[0]
df = df[df['Attack'] <= 200]
print(f'发现{attack_anomaly_count}条 Attack 属性大于 200 的异常数据，已删除。')

# 7. 查找并纠正 Generation 与 Legendary 属性被置换的两条数据，同时处理这两列中其他有问题的数据
# 假设 Generation 应该是整数，Legendary 应该是布尔值或类似的值
wrong_rows = df[~df['Generation'].astype(str).str.isdigit() & df['Legendary'].astype(str).str.isdigit()]
if not wrong_rows.empty:
    for index in wrong_rows.index:
        df.at[index, 'Generation'], df.at[index, 'Legendary'] = df.at[index, 'Legendary'], df.at[index, 'Generation']
    print('已纠正两条Generation和Legendary属性被置换的数据。')
else:
    print('不存在 Generation 与 Legendary 属性被置换的数据。')

# 处理 Generation 列中不是整数的数据
generation_anomaly_count = df[~df['Generation'].astype(str).str.isdigit()].shape[0]
df = df[df['Generation'].astype(str).str.isdigit()]
print(f'发现{generation_anomaly_count}条 Generation 列不是整数的异常数据，已删除。')

# 处理 Generation 列中小于 1 的数据
df['Generation'] = pd.to_numeric(df['Generation'])
generation_less_than_one_count = df[df['Generation'] < 1].shape[0]
df = df[df['Generation'] >= 1]
print(f'发现{generation_less_than_one_count}条 Generation 列小于 1 的异常数据，已删除。')

# 处理 Legendary 列中不是 TRUE 或 FALSE 的数据
valid_legendary_values = ['TRUE', 'FALSE']
legendary_anomaly_count = df[~df['Legendary'].astype(str).isin(valid_legendary_values)].shape[0]
df = df[df['Legendary'].astype(str).isin(valid_legendary_values)]
print(f'发现{legendary_anomaly_count}条 Legendary 列取值非 TRUE 或 FALSE 的异常数据，已删除。')

# 8. 处理 Defense 列中的负值异常数据
defense_negative_count = df[df['Defense'] < 0].shape[0]
df = df[df['Defense'] >= 0]
print(f'发现{defense_negative_count}条 Defense 列的负值异常数据，已删除。')

# 9. 处理 Sp. Atk 列中超过 200 的异常数据
sp_atk_anomaly_count = df[df['Sp. Atk'] > 200].shape[0]
df = df[df['Sp. Atk'] <= 200]
print(f'发现{sp_atk_anomaly_count}条 Sp. Atk 列超过 200 的异常数据，已删除。')

# 10. 处理 Speed 列中的负值异常数据
speed_negative_count = df[df['Speed'] < 0].shape[0]
df = df[df['Speed'] >= 0]
print(f'发现{speed_negative_count}条 Speed 列的负值异常数据，已删除。')

# 11. 检查 Total 列的值是否为 HP、Attack、Defense、Sp. Atk、Sp. Def、Speed 这六列的和，如有误则进行修改
df[['HP', 'Defense', 'Sp. Def', 'Speed']] = df[['HP', 'Defense', 'Sp. Def', 'Speed']].apply(pd.to_numeric, errors='coerce')

# 找到参与计算的六列数据中小数点后位数最多的值
max_decimals = max(df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].apply(lambda x: x.astype(str).str.split('.').str[1].str.len().max()).fillna(0))

# 计算 Total 列并按最多小数点位数保留
df['Total'] = (df['HP'] + df['Attack'] + df['Defense'] + df['Sp. Atk'] + df['Sp. Def'] + df['Speed']).round(max_decimals)
print(f'已重新计算 Total 列的值，确保其准确性。')

# 重新给 # 列按数字顺序编写
df['#'] = range(1, len(df) + 1)

# 将结果保存为 csv 文件
csv_path = 'Pokemon_cleaned.csv'
df.to_csv(csv_path, index=False)

删除了最后两行无意义数据。
发现4条除 Type 2 列和 Total 列外其他列存在空白值的异常数据，已删除。
发现6条重复数据，已删除。
发现4条 Type 2 列的异常数据，已将异常值清空。
发现2条 Attack 属性大于 200 的异常数据，已删除。
已纠正两条Generation和Legendary属性被置换的数据。
发现1条 Generation 列不是整数的异常数据，已删除。
发现1条 Generation 列小于 1 的异常数据，已删除。
发现4条 Legendary 列取值非 TRUE 或 FALSE 的异常数据，已删除。
发现1条 Defense 列的负值异常数据，已删除。
发现1条 Sp. Atk 列超过 200 的异常数据，已删除。
发现1条 Speed 列的负值异常数据，已删除。
已重新计算 Total 列的值，确保其准确性。
